In [2]:
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

We follow the procedure outlined in 3.1.2.1 (Experiment 1)

Currently: 

* $n = 12$
* $k = 8$
* $p \in \{1, 2, 4, 8\}$
* training instances = $100$
* evaluation instances = $500$


Step 1: Generation of $\underline{\beta}, \underline{\gamma} \in \mathbb{R}^p$

In [3]:
from benchmark.random_k_sat import RandomKSAT

# Read in 100 randomly generated CNF(12, 8, r_8) instances
num_instances = 100
n = 12
k = 8

instances = RandomKSAT.from_poisson(n=n, k=k, instances=num_instances, from_file=True, calc_naive=True)

In [3]:
import torch
from k_sat.pytorch_solver.pytorch_circuit import PytorchCircuit
from k_sat.pytorch_solver.pytorch_optimiser import PytorchOptimiser

ps = [1, 2, 4, 8, 16] #[1, 2, 4, 8, 30, 60]
optimal_params = {}

for p in ps:
	print(f'Training for p = {p}')
	circuit = PytorchCircuit(num_vars=n, layers=p)
	adam = torch.optim.Adam(circuit.parameters(), lr=0.1, maximize=True)
	optimiser = PytorchOptimiser(circuit, adam, epochs=40)
	optimiser.find_optimal_params(instances.formulas)
	optimal_params[p] = (circuit.gamma.detach().clone(), circuit.beta.detach().clone())

Training for p = 1
Epoch 0, p_succ: 0.0004537288914434612
Epoch 10, p_succ: 0.0050414674915373325
Epoch 20, p_succ: 0.005677502136677504
Epoch 30, p_succ: 0.00569401727989316
Epoch 40, p_succ: 0.005692856851965189
Training for p = 2
Epoch 0, p_succ: 0.0004627828311640769
Epoch 10, p_succ: 0.013224032707512379
Epoch 20, p_succ: 0.015511535108089447
Epoch 30, p_succ: 0.016067299991846085
Epoch 40, p_succ: 0.016087966039776802
Training for p = 4
Epoch 0, p_succ: 0.0004948183777742088
Epoch 10, p_succ: 0.03856354206800461
Epoch 20, p_succ: 0.046166371554136276
Epoch 30, p_succ: 0.04802190884947777
Epoch 40, p_succ: 0.04795888066291809


In [10]:
optimal_params

{1: (tensor([-0.4660]), tensor([0.3218])),
 2: (tensor([-0.3617, -0.5444]), tensor([0.3608, 0.2734])),
 4: (tensor([-0.2743, -0.4393, -0.4892, -0.5560]),
  tensor([0.3621, 0.3323, 0.2928, 0.2067])),
 8: (tensor([-0.1943, -0.3142, -0.3775, -0.4085, -0.4177, -0.4403, -0.4680, -0.5495]),
  tensor([0.3702, 0.3314, 0.3098, 0.2827, 0.2594, 0.2351, 0.2175, 0.1585])),
 16: (tensor([-0.1310, -0.2246, -0.2878, -0.3188, -0.3206, -0.3239, -0.3589, -0.3933,
          -0.3972, -0.4185, -0.4225, -0.4220, -0.4057, -0.4495, -0.5107, -0.5579]),
  tensor([0.3828, 0.3519, 0.3277, 0.2958, 0.2579, 0.2579, 0.2734, 0.2709, 0.2575,
          0.2592, 0.2381, 0.2027, 0.1893, 0.1784, 0.1471, 0.0879]))}

In [ ]:
import h5py
# Save to file
parent_dir = os.path.dirname(os.getcwd())
dir = f"{parent_dir}/benchmark/instances/n_{n}"
with h5py.File(f'{dir}/a_params.hdf5', 'w') as file:
	for p in ps:
		file.create_dataset(f'{p}_g', data=optimal_params[p][0])
		file.create_dataset(f'{p}_b', data=optimal_params[p][1])

Step 2: Evaluate on satisfiable CNF instances

In [4]:
import h5py
import torch

n = 12
ps = [8, 16]

# Read params in from file
parent_dir = os.path.dirname(os.getcwd())
dir = f"{parent_dir}/benchmark/instances/n_{n}"
optimal_params_r = {}
with h5py.File(f'{dir}/a_params.hdf5', 'r') as file:
	for p in ps:
		gamma = torch.from_numpy(file.get(f'{p}_g')[:])
		beta = torch.from_numpy(file.get(f'{p}_b')[:])
		optimal_params_r[p] = (gamma, beta)

In [8]:
from k_sat.pytorch_solver.pytorch_circuit import PytorchCircuit

k_e = k
n_e = [12] #[i in range(12, 21)]
num_instances_e = 1000

p_succ = {}

for n in n_e:
	print(f'evaluating n = {n}')

	# Read in random problems
	instances_e = RandomKSAT.from_poisson(n=n, k=k, instances=num_instances_e, from_file=num_instances, calc_naive=True)

	for (p, params) in optimal_params_r.items():
		print(f'evaluating p = {p}')

		# Initialise QAOA circuit
		circuit = PytorchCircuit(n, p, params[0], params[1])

		# Evolve each instance and take mean of p_succ
		prob = 0
		for instance in instances_e.formulas:
			prob += circuit(instance.naive_counts, instance.naive_sats).item()
		
		p_succ[p] = prob/num_instances_e

evaluating n = 12
evaluating p = 8
evaluating p = 16


In [9]:
p_succ

{8: 0.12876187145031145, 16: 0.286059484512778}